In [ ]:
READ ME

1. /etc/httpd/conf.d 에서
README  autoindex.conf  userdir.conf  welcome.conf
외에 나머지 vhost 파일 (ex. heeyoung.conf, vhost.conf)이 있다면,
각각 들어가셔서 Port와 ServerName이 일치하는게 있는지 확인하세요.

ex. 1  => 오류 발생
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName ns.test.com

<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName ns.test.com

ex. 2  => 오류 없음!
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName ns.test.com

<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName ww.test.com
포트가 같아도 상관은 없지만 이 경우에 ServerName이 달라야합니다!
어느 vhost 파일도 ServerName과 Port번호가 동시에 일치해 중복되서는 안됩니다.

ex. 3 => 오류 발생
ServerName을 생략해도 괜찮지만 그런 경우에는 어느 vhost 파일도 Port가 동시에 일치해 중복되선 안됩니다.
<VirtualHost *:80>
    ServerAdmin webmaster@localhost

<VirtualHost *:80>
    ServerAdmin webmaster@localhost

2. 나중에 DNS를 만들게 된다면 ServerName을 zone 파일과 일치시켜야합니다.

*** 문제가 발생하거나 이해가 안되면 꼭 저에게 물어봐주세요. ***


In [ ]:
import paramiko
import time
import sys

ip = "172.16.254.12"
user = "root"
pw = "asd123!@"

# 고정된 경로
HTTPD_CONF = ""
VHOST_DIR = ""

# 동적으로 받을 값
WEB_DOCUMENT_ROOT = ""
VHOST_CONF = ""
SERVER_NAME = ""
ERROR_LOG = ""
CUSTOM_LOG = ""


def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def set_firewalld():
    cc("systemctl stop firewalld; systemctl disable firewalld")

def install_httpd():
    cc("dnf -y install httpd ; systemctl enable --now httpd")
    cc("dnf install expect -y")

def set_user_dir_and_vhost_conf_rocky():
    global WEB_DOCUMENT_ROOT
    WEB_DOCUMENT_ROOT = input("사용할 웹 디렉토리를 입력하세요 (예: /home/team1 또는 /var/www/young): ").strip()
    cc(f"mkdir -p {WEB_DOCUMENT_ROOT} && chmod 755 {WEB_DOCUMENT_ROOT}")
    print(f" 웹 문서 루트: {WEB_DOCUMENT_ROOT}")

def set_vhostconf_rocky():
  #  VirtualHost 설정 -> 가상 호스트 파일이 있으면 우선 적용이기 떄문에 
  # /etc/httpd/conf/httpd.conf의 124, 129,136줄을 수정하지 않아도 됩니다.
    
    global VHOST_CONF, SERVER_NAME, ERROR_LOG, CUSTOM_LOG
    vhost_name = input("VHost 설정 파일을 만들겠습니다 원하는 이름을 입력하세요 (예: vhost.conf): ")
    
    SERVER_NAME = input("ServerName을 입력하세요. 주의!! 나중에 dns를 만들때 도메인 네임과 일치시키지 않으면 문제발생 (예: ns.test.com): ")
    f = input("ErrorLog 경로를 입력하세요 (예: /var/log/httpd/test_error.log): ")
    CUSTOM_LOG = input("CustomLog 경로를 입력하세요 (예: /var/log/httpd/test_access.log): ")

    
    VHOST_CONF = f"/etc/httpd/conf.d/{vhost_name}"
    print(f" {VHOST_CONF} 생성 중...")
    cmd = f"""
cat <<EOF > "{VHOST_CONF}"
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName {SERVER_NAME}
    DocumentRoot {WEB_DOCUMENT_ROOT}
    
    <Directory {WEB_DOCUMENT_ROOT}>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

    ErrorLog {ERROR_LOG}
    CustomLog {CUSTOM_LOG} combined
</VirtualHost>
EOF
"""
    cc(cmd)

def mk_index():
    """기본 index.html 생성"""
    print(" index.html 생성 중...")
    cmd = f"echo '<h1>This is index.html for Test by mk_index func</h1>' > {WEB_DOCUMENT_ROOT}/index.html"
    print(f"{WEB_DOCUMENT_ROOT} 경로에 파일이 생성되었습니다")
    cc(cmd)

def restart_httpd():
    cc("systemctl restart httpd")
    print("Apache 재시작 완료!")

def install_vsftpd():
    print("vsftpd 설치 중...")
    cc("dnf -y install vsftpd")
    cc("systemctl enable --now vsftpd")
    print("systemctl status vsftpd")

def add_ftpuser():
    ftpuser = input("ftp user 이름을 만듭니다. 이름을 입력해주세요. (ex. ftpuser): ")
    
    # 사용자 없으면 생성
    cc(f"id -u {ftpuser} >/dev/null 2>&1 || useradd -m {ftpuser}")
    
    # Expect 스크립트 파일 생성
    cmd = f"""
cat <<EOF > /root/mk_ftpuser.exp
#!/usr/bin/expect -f
set timeout -1
spawn passwd {ftpuser}
expect "New password:"
send "asd123!@\\r"
expect "Retype new password:"
send "asd123!@\\r"
expect eof
EOF
"""
    cc(cmd)
    
    # 실행 권한 부여 후 실행
    cc("chmod +x /root/mk_ftpuser.exp")
    print(cc("/usr/bin/expect /root/mk_ftpuser.exp"))


def go():
    set_firewalld()
    install_httpd()
    set_user_dir_and_vhost_conf_rocky()
    set_vhostconf_rocky()
    mk_index()
    restart_httpd()
    install_vsftpd()
    add_ftpuser()

go()

Failed to stop firewalld.service: Unit firewalld.service not loaded.
Failed to disable unit: Unit file firewalld.service does not exist.

bash: line 1: dnf: command not found
Failed to enable unit: Unit file httpd.service does not exist.

bash: line 1: dnf: command not found

